In [1]:
import os
import math
import torch
import wandb
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.utilities.model_summary import ModelSummary
from torchvision.transforms import v2

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from models.model_cnn import KaninchenModel, KaninchenModelResidual, KaninchenModel_v1, KaninchenModel_v2, KaninchenModel_v3, KaninchenModel_v4, KaninchenModel_v5
from data.datamodule import BinaryImageDataModule, ReducedSizeBinaryImageDataModule

import optuna
from training.hyperparameter_tuning import CnnOptunaTrainer

c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Loading Configuration

In the following steps, we will load the configuration settings using the `load_configuration` function. The configuration is stored in the `config` variable which will be used throughout the script.

In [2]:
from config.load_configuration import load_configuration
config = load_configuration()

PC Name: DESKTOP-LUKAS
Loaded configuration from config/config_lukas.yaml


### Logging in to Weights & Biases (wandb)

Before starting any experiment tracking, ensure you are logged in to your Weights & Biases (wandb) account. This enables automatic logging of metrics, model checkpoints, and experiment configurations. The following code logs you in to wandb:

```python
wandb.login()
```
If you are running this for the first time, you may be prompted to enter your API key.

In [3]:
# Initialize the Wandb logger
wandb.login()

wandb: Currently logged in as: lukas-pelz (HKA-EKG-Signalverarbeitung) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

### Setting Seeds for Reproducibility

To ensure comparable and reproducible results, we set the random seed using the `seed_everything` function from PyTorch Lightning. This helps in achieving consistent behavior across multiple runs of the notebook.

In [4]:
pl.seed_everything(config['seed'])
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"   # disable oneDNN optimizations for reproducibility

Seed set to 42


### Checking for GPU Devices

In this step, we check for the availability of GPU devices and print the device currently being used by PyTorch. This ensures that the computations are performed on the most efficient hardware available.

In [5]:
# Check if CUDA is available and set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('Torch Version: ', torch.__version__)
print('Using device: ', device)
if device.type == 'cuda':
    print('Cuda Version: ', torch.version.cuda)
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
    torch.set_float32_matmul_precision('high')

Torch Version:  2.7.0+cu128
Using device:  cuda
Cuda Version:  12.8
NVIDIA GeForce RTX 5060 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


### Defining Transformations and Instantiating DataModule

In this step, we will define the necessary data transformations and initialize the `Animal_DataModule` with the provided configuration.

In [6]:
# TODO: Define transformations here
import os

size = config['image_size']

import data.custom_transforms as custom_transforms
transform = v2.Compose([
    custom_transforms.CenterCropSquare(),
    v2.Resize((size, size)),
    v2.ToTensor(),
    # v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dm = ReducedSizeBinaryImageDataModule(data_dir=config['path_to_split_aug_pics'], transform=transform, batch_size=config['batch_size'], num_workers=6) #, persistent_workers=True)
# dm = BinaryImageDataModule(data_dir=config['path_to_split_aug_pics'], transform=transform, batch_size=config['batch_size'], num_workers=2, persistent_workers=True)
# dm.setup()

# train_loader = dm.train_dataloader()
# val_loader = dm.val_dataloader()
# test_loader = dm.test_dataloader()

# # Show a few images from the training set
# from torchvision.utils import make_grid
# def show_images(loader):
#     images, labels = next(iter(loader))
#     images = images[:16]  # Show only the first 16 images
#     labels = labels[:16]
#     grid = make_grid(images, nrow=4, padding=2)
#     plt.figure(figsize=(10, 10))
#     plt.imshow(grid.permute(1, 2, 0).numpy())
#     plt.title('Sample Images')
#     plt.axis('off')
#     plt.show()
# show_images(train_loader)

# print('Train dataset size:', len(dm.train_dataset))
# print('Validation dataset size:', len(dm.val_dataset))
# print('Test dataset size:', len(dm.test_dataset))

c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


### Creating the Model

In this step, we will define the model architecture and print its summary using the `ModelSummary` utility from PyTorch Lightning. This provides an overview of the model's layers, parameters, and structure.

In [ ]:
#model = CatsDogsModel()
# model = KaninchenModel()
model = KaninchenModel_v1()
print(ModelSummary(model, max_depth=-1))  
print(type(model).__name__)

### Training the Model and Logging with Weights & Biases

In this step, we initialize the Wandb logger and configure the experiment name to include a timestamp for better tracking. The `Trainer` from PyTorch Lightning is set up with the Wandb logger and an early stopping callback to monitor validation loss and prevent overfitting. After training, the Wandb run is finished, and the trained model checkpoint is saved with a unique filename containing the current date and time.

In [ ]:
# Initialize the Wandb logger
# add time to the name of the experiment
import datetime
now = datetime.datetime.now()
current_time = now.strftime("%Y-%m-%d_%H-%M-%S")

# Initialize wandb logger
wandb_logger = WandbLogger(
    project=config['wandb_project_name'],
    name=f"{config['wandb_experiment_name']}_{type(model).__name__}_{current_time}",
    config={
        'model': type(model).__name__,
        'dataset': 'DwarfRabbits-binary',
        'batch_size': config['batch_size'],
        'max_epochs': config['max_epochs'],
        'learning_rate': config['learning_rate']
    }
)

# Initialize Trainer with wandb logger, using early stopping callback (https://lightning.ai/docs/pytorch/stable/common/early_stopping.html)
trainer = Trainer(
    max_epochs=config['max_epochs'], 
    default_root_dir='model/checkpoint/', #data_directory, 
    accelerator="auto", 
    devices="auto", 
    strategy="auto",
    callbacks=[EarlyStopping(monitor='val_loss', patience=5, mode='min')], 
    logger=wandb_logger)

# Training of the model
trainer.fit(model=model, datamodule=dm)

# Finish wandb
wandb.finish()

# Create a filename with date identifier
model_filename = f"{config['wandb_experiment_name']}_{type(model).__name__}_{current_time}.ckpt"

# Save the model's state_dict to the path specified in config
save_path = os.path.join(os.path.dirname(config['path_to_models']), model_filename)
trainer.save_checkpoint(save_path)
print(f"Model checkpoint saved as {save_path}")
config['path_to_model'] = save_path

In [ ]:
# List of all model classes to train
model_classes = [KaninchenModel_v1, KaninchenModel_v2, KaninchenModel_v3, KaninchenModel_v4, KaninchenModel_v5, KaninchenModelResidual]

import datetime
for model_class in model_classes:
    # Create model instance
    model = model_class()
    print(ModelSummary(model, max_depth=-1))  
    print(type(model).__name__)

    # Initialize the Wandb logger
    # add time to the name of the experiment
    now = datetime.datetime.now()
    current_time = now.strftime("%Y-%m-%d_%H-%M-%S")

    # Initialize wandb logger
    wandb_logger = WandbLogger(
        project=config['wandb_project_name'],
        name=f"{config['wandb_experiment_name']}_{type(model).__name__}_{current_time}",
        config={
            'model': type(model).__name__,
            'dataset': 'DwarfRabbits-binary',
            'batch_size': config['batch_size'],
            'max_epochs': config['max_epochs'],
            'learning_rate': config['learning_rate']
        }
    )

    # Initialize Trainer with wandb logger, using early stopping callback
    trainer = Trainer(
        max_epochs=config['max_epochs'], 
        default_root_dir='model/checkpoint/',
        accelerator="auto", 
        devices="auto", 
        strategy="auto",
        callbacks=[EarlyStopping(monitor='val_loss', patience=5, mode='min')], 
        logger=wandb_logger)

    # Training of the model
    trainer.fit(model=model, datamodule=dm)

    # Finish wandb
    wandb.finish()

    # Create a filename with date identifier
    model_filename = f"{config['wandb_experiment_name']}_{type(model).__name__}_{current_time}.ckpt"

    # Save the model's state_dict to the path specified in config
    save_path = os.path.join(os.path.dirname(config['path_to_models']), model_filename)
    trainer.save_checkpoint(save_path)
    print(f"Model checkpoint saved as {save_path}")
    
    # Update config with the last trained model path
    config['path_to_model'] = save_path
    
    print(f"Completed training for {model_class.__name__}")

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


   | Name                | Type              | Params | Mode 
-------------------------------------------------------------------
0  | criterion           | BCEWithLogitsLoss | 0      | train
1  | sigmoid             | Sigmoid           | 0      | train
2  | model               | Sequential        | 1.1 M  | train
3  | model.0             | Conv2d            | 896    | train
4  | model.1             | BatchNorm2d       | 64     | train
5  | model.2             | ReLU              | 0      | train
6  | model.3             | MaxPool2d         | 0      | train
7  | model.4             | Conv2d            | 18.5 K | train
8  | model.5             | BatchNorm2d       | 128    | train
9  | model.6             | ReLU              | 0      | train
10 | model.7             | MaxPool2d         | 0      | train
11 | model.8             | Conv2d            | 73.9 K | train
12 | model.9             | BatchNorm2d       | 256    | train
13 | model.10            | ReLU              | 0      | train
14

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name           | Type              | Params | Mode 
--------------------------------------------------------------
0  | criterion      | BCEWithLogitsLoss | 0      | train
1  | sigmoid        | Sigmoid           | 0      | train
2  | model          | Sequential        | 1.1 M  | train
3  | train_accuracy | BinaryAccuracy    | 0      | train
4  | val_accuracy   | BinaryAccuracy    | 0      | train
5  | val_precision  | BinaryPrecision   | 0      | train
6  | val_recall     | BinaryRecall      | 0      | train
7  | test_accuracy  | BinaryAccuracy    | 0      | train
8  | init_conv      | Sequential        | 3.8 K  | train
9  | layer1         | Sequential        | 295 K  | train
10 | layer2         | Sequential        | 886 K  | train
11 | layer3         | Sequential        | 1.2 M  | train
12 | layer4         | Sequential        | 3.5 M  | train
13 | layer5         | ResidualBlock     | 5.0 M  | train
14 | pool           | AdaptiveAvgPool2d

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric BinaryAccuracy was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)
c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric BinaryPrecision was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)
c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric BinaryRecall was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)
c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\pytorch_lightning\trainer\connecto

Epoch 13: 100%|██████████| 161/161 [00:50<00:00,  3.17it/s, v_num=je5w, val_loss=0.536]


Validation Data ROC AUC,▁▄▇▇▅▆▄▇▇██▇█▆█
epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇███
train_acc,▁▂▃▃▄▄▅▅▆▆▇▇██
train_loss,▅▇▆▄▅▅▅▆▅▆▇▃▂▅█▅▂▄▃▄▃▅▄▃▆▄▄▄▂▂▃▃▄▃▁▂▁▄▃▂
trainer/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇█████
val_acc,▁▇▆▆▇▄▇▄█▇▇█▇▇
val_loss,▄▂▂▂▂█▁▄▁▂▂▂▅▂
val_precision,▁▅▄▅▆▃▆▃▇▅▆▆█▅
val_recall,█▄▅▂▃▅▄█▄▇▅▇▁▆
Validation Data ROC AUC,0.8136
epoch,13


Model checkpoint saved as C:\Users\lukas\SynologyDrive_IMS/SS25_MSYS_KAER-AI-PoseAct/21_Test_Data/Models/CNN\binaryClassification_CNN_KaninchenModel_v2_2025-06-02_19-28-58.ckpt
Completed training for KaninchenModel_v2


### Hyperparameter Optimization with Optuna

This section performs automated hyperparameter tuning using Optuna to find the optimal model configuration. The optimization process searches for hyperparameters that minimize validation loss across multiple trials, helping to improve model performance beyond manual tuning.

In [ ]:
# Hyperparameter optimization
import datetime
config['sweep_id'] = datetime.datetime.now().strftime("%Y%m%d_%H%M")

def objective(trial):
    model = KaninchenModel                  # or another model's class, depending on your choice
    trainer = CnnOptunaTrainer(
        model=model,                        # Function to create the model
        config=config,
        normalize_mean=None, #[0.485, 0.456, 0.406], 
        normalize_std=None, #[0.229, 0.224, 0.225],
        dataset_name="DwarfRabbits-binary"
    )
    return trainer.run_training(trial)

# Create an Optuna study
study = optuna.create_study(direction="minimize")  # because we minimize val_loss

# Set verbosity to WARNING to reduce output clutter
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Start the hyperparameter optimization
study.optimize(objective, n_trials=config['number_of_trials'])
# study.optimize(objective, n_trials=3)

# Best result
print("Best trial:")
print(study.best_trial.params)
print("Best value (val_loss):", study.best_value)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

# Predict with the Model


In [ ]:
# from PIL import Image
# import torch
# # Load the saved model weights from the path specified in config

# def predict_image(path, model):
#     transform = transforms.Compose([
#         transforms.Resize((150, 150)),
#         transforms.ToTensor(),
#         transforms.Normalize([0.5]*3, [0.5]*3)
#     ])

#     img = Image.open(path).convert('RGB')
#     img = transform(img).unsqueeze(0)  # Add batch dimension

#     model.eval()
#     with torch.no_grad():
#         pred = model(img)
#         result = "Dog" if pred.item() > 0.5 else "Cat"
#     print(f"Prediction: {result}")


### Loading and Evaluating the Trained Model

The trained model is loaded from the checkpoint specified in the configuration. If the checkpoint exists, the model weights are restored and the model is set to evaluation mode. PyTorch Lightning's `Trainer` is then used to evaluate the model on the test dataset, providing a streamlined way to assess model performance after training.

In [ ]:
model_path = config['path_to_model']
if model_path and os.path.exists(model_path):
    #model = CatsDogsModel.load_from_checkpoint(model_path, map_location=device)
    model = KaninchenModel.load_from_checkpoint(model_path, map_location=device)
    print(f"Loaded model weights from {model_path}")
else:
    print("Model path not found or not specified in config.")

# Ensure model is in eval mode
model.eval()

# Pytorch Lightning's Trainer can be used to test the model
trainer = Trainer()
trainer.test(model=model, dataloaders=test_loader)